This model works to estimate the prices based on the following features:

In [3]:
#defining imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
from keras import layers


Matplotlib is building the font cache; this may take a moment.


In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
